# This is the notebook for the project of Applied Data Analysis.

### This project is based on the data of the openfood dataset found on https://world.openfoodfacts.org/data 

The goal of this notebook is the following:

1) Learn to handle the datas that we use

2) Understand the format of the datas (and also clean them)

3) Filter and transform the datas


#### First we need to import every library we will need and import the datas

In [1]:
# Put your imports here.
import os
import pandas as pd
import json
import folium
import numpy as np
from datetime import datetime, date, time
import dateutil.parser

# WARNING : Make sure that Seaborn is updated (0.9.0) on your computer !
import seaborn as sns

Now we load the datas from the csv file

In [2]:
DATAPATH = "./data/"

In [3]:
food = pd.read_csv(DATAPATH+ "openfoodfacts.csv", sep='\t', low_memory = False)

In [4]:
food_cleaned = food.copy()

In [5]:
food_cleaned.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1529059204,2018-06-15T10:40:04Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000000000018,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369398,2018-11-16T11:56:38Z,1542369398,2018-11-16T11:56:38Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000000000019,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369403,2018-11-16T11:56:43Z,1542369403,2018-11-16T11:56:43Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
food_cleaned.dropna(how="all",inplace = True)
food_cleaned.dropna(how="all",axis=1,inplace=True)

We can also separate the table into two parts. We can split the food properties (energy and others) from the rest of the table. This way we will only handle a smaller table containing the values that we need. And if needed later on, we can check the values of the nutriments in the other table.

The columns that are concerned finishes with "_100g"

In [7]:
Col_nutriments = []
for c in food_cleaned.columns:
    if c.endswith("_100g"):
        Col_nutriments.append(c)
Col_nutriments
nutriments = food_cleaned[Col_nutriments]

In [8]:
food_cleaned.drop(columns=Col_nutriments,inplace=True)

In [9]:
food_cleaned = food_cleaned[pd.notnull(food_cleaned['created_datetime'])].reset_index()

In [10]:
food_cleaned.head()

,index,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1529059204,2018-06-15T10:40:04Z,Vitória crackers,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,000000000018,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369398,2018-11-16T11:56:38Z,1542369398,2018-11-16T11:56:38Z,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,000000000019,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369403,2018-11-16T11:56:43Z,1542369403,2018-11-16T11:56:43Z,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First we notice that we can remove the column code since the `code` can be found in the end of each url in the column `url`. First check that this is correct:

In [11]:
url = food_cleaned.url.apply(lambda s: str(s).split("/"))

In [12]:
new_url = []
indices_bad = []
for i in range(len(url)):
    try:
        new_url.append(url[i][4])
    except:
        indices_bad.append(i)
        continue
len(new_url)

695172

In [13]:
food_cleaned.created_datetime[indices_bad]

264659                                 Magasins U,Carrefour
264712                                               Auchan
264778                                          Dia,Sogeres
264794                                              Super U
264818                                               Auchan
264831                                            Carrefour
264840                                               Auchan
264892    Banque alimentaire,Carrefour,Leclerc,E.Leclerc...
264912                                 Franprix,Intermarché
264922                                     Carrefour market
264924                                     Carrefour market
264929                                       Super U,Lecler
264940                                                 Spar
264942                                   Banque alimentaire
264949                                              Leclerc
264952                                       Auchan,Super U
264988                                  

We notice that some entries are completly wrong, so we will remove them...

In [14]:
food_cleaned.drop(indices_bad, inplace=True)

Now, we can compare the urls that we have with the code, in order to see if the assumption that we made is correct

In [15]:
code = food_cleaned.code.values

In [16]:
new_url = np.array(new_url)

In [17]:
np.sum(new_url == code) == len(code) # Check if is only true value, so the sum must be equal to the length of the arrays

True

Great, so now we can safely remove the column `code` since all the information about the code can be found in the url!

In [18]:
food_cleaned.drop(columns = ['code'],inplace=True)

In [19]:
# We also remove the index column
food_cleaned.drop(columns = ['index'],inplace=True)
food_cleaned.head()

,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1529059204,2018-06-15T10:40:04Z,Vitória crackers,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369398,2018-11-16T11:56:38Z,1542369398,2018-11-16T11:56:38Z,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369403,2018-11-16T11:56:43Z,1542369403,2018-11-16T11:56:43Z,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We now need to transform the columns containing time. To do so, we know (from the description of the columns given on openfoodfacts) that the columns starting ending with a certain suffix are the one containing dates:

1) datetime : yyyy-mm-ddThh:mn:ssZ

2) t : unix timestamp

So we first need to find all the columns that respect one of the above criteria and transform those columns to the right format

In [20]:
food_cleaned.created_datetime.values

array(['2018-06-15T10:38:00Z', '2018-10-13T21:06:14Z',
       '2018-08-31T17:53:02Z', ..., '2018-10-15T09:55:58Z',
       '2018-02-20T17:07:29Z', '2018-10-06T09:33:52Z'], dtype=object)

In [21]:
# Find the columns whose name finishes by "_datetime"
col_datetime = [c for c in food_cleaned.columns if c.endswith("_datetime")]
# Same for the columns whose name finishes by "_t"
col_t = [c for c in food_cleaned.columns if c.endswith("_t")]
#food_datetime = food_cleaned[col_datetime]
#food_t = food_cleaned[col_t]

First we modify the datetime column, using the strptime method.

In [22]:
food_cleaned[col_datetime[0]] = food_cleaned[col_datetime[0]].apply(lambda d: datetime.strptime(d,"%Y-%m-%dT%H:%M:%SZ"))

In [23]:
food_cleaned[col_datetime[1]] = food_cleaned[col_datetime[1]].apply(lambda d: datetime.strptime(d,"%Y-%m-%dT%H:%M:%SZ"))

In [24]:
food_cleaned.head()

,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15 10:38:00,1529059204,2018-06-15 10:40:04,Vitória crackers,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13 21:06:14,1539464817,2018-10-13 21:06:57,Cacao,NaN,130 g,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31 17:53:02,1535737986,2018-08-31 17:53:06,Sauce Sweety chili 0%,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369398,2018-11-16 11:56:38,1542369398,2018-11-16 11:56:38,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1542369403,2018-11-16 11:56:43,1542369403,2018-11-16 11:56:43,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we need to take care of the unix format

In [25]:
food_cleaned[col_t[0]] = pd.to_datetime(food_cleaned[col_t[0]],unit='s')

In [26]:
food_cleaned[col_t[1]] = pd.to_datetime(food_cleaned[col_t[1]],unit='s')

In [27]:
food_cleaned.head()

,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-06-15 10:38:00,2018-06-15 10:38:00,2018-06-15 10:40:04,2018-06-15 10:40:04,Vitória crackers,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,2018-10-13 21:06:14,2018-10-13 21:06:14,2018-10-13 21:06:57,2018-10-13 21:06:57,Cacao,NaN,130 g,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-08-31 17:53:02,2018-08-31 17:53:02,2018-08-31 17:53:06,2018-08-31 17:53:06,Sauce Sweety chili 0%,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:38,2018-11-16 11:56:38,2018-11-16 11:56:38,2018-11-16 11:56:38,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:43,2018-11-16 11:56:43,2018-11-16 11:56:43,2018-11-16 11:56:43,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We notice that the date in Unix format and the other one are exactly the same, so we can get rid of one of the two.
This will again reduce the size of the dataframe, and will be easier to handle

In [28]:
# Verify that it is indeed the same
np.sum(food_cleaned.created_t.values == food_cleaned.created_datetime.values) == len(food_cleaned.created_t.values)

True

In [29]:
# Verify that it is indeed the same
np.sum(food_cleaned.last_modified_t.values == food_cleaned.last_modified_datetime.values) == len(food_cleaned.last_modified_t.values)

True

In [30]:
# We drop the "_datetime" (arbitrary choice)
food_cleaned.drop(columns = ['created_datetime'],inplace=True)
food_cleaned.drop(columns = ['last_modified_datetime'],inplace=True)

In [31]:
food_cleaned.head()

,url,creator,created_t,last_modified_t,product_name,generic_name,quantity,packaging,packaging_tags,brands,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-06-15 10:38:00,2018-06-15 10:40:04,Vitória crackers,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,2018-10-13 21:06:14,2018-10-13 21:06:57,Cacao,NaN,130 g,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-08-31 17:53:02,2018-08-31 17:53:06,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:38,2018-11-16 11:56:38,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:43,2018-11-16 11:56:43,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We now need to check if there are columns that contains only `NaN`values in it. 

In [32]:
col_na = []
for col in food_cleaned:
    if(food_cleaned[str(col)].isna().all()):
        col_na.append(str(col))
col_na

['allergens_en', 'no_nutriments']

There are still two columns that contains only `NaN` values in it, so we can remove them two.

In [33]:
food_cleaned.drop(columns=col_na,inplace=True)

In [34]:
food_cleaned.head()

,url,creator,created_t,last_modified_t,product_name,generic_name,quantity,packaging,packaging_tags,brands,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-06-15 10:38:00,2018-06-15 10:40:04,Vitória crackers,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,2018-10-13 21:06:14,2018-10-13 21:06:57,Cacao,NaN,130 g,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-08-31 17:53:02,2018-08-31 17:53:06,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:38,2018-11-16 11:56:38,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:43,2018-11-16 11:56:43,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we clean part of the table. We still need to know what we want to do with na values in this table.

1) We can replace them all by 0. This is not necessary a good solution since some of those Na values are string and some are floats.

2) We can replace the floats na values with a 0 and the string with a empty space.

The last solution is better since it takes into account the type of the column.

In [35]:
# Do this part

At this point, we have two tables that are cleaned (one containing the nutriment informations),the other contains all the rest of the table. We removed the columns that contains only `NaN` values and removed the columns that are redudant, as for example the time of creation and last modification and also the column for the bar-code, since this value can be found in the url of the product. 

Let's check out the columns that we can remove, not because they don't contain information but because the information inside is not useful

In [83]:
# The first we check is the "generic_name" column
print((len(food_cleaned.generic_name)-np.sum(food_cleaned.generic_name.isnull()))/len(food_cleaned.generic_name))
for i in range(len(food_cleaned.generic_name)):
    if (not (pd.isnull(food_cleaned.generic_name[i]))):
        print(food_cleaned.product_name[i])
        print(food_cleaned.generic_name[i])
        print("===========")
# The problem with the generic name is that there is a lot of values that are not well-written. 
# Some people didn't undertand how to fill this value... 
# Plus only 11 percent of the total values are different than nan values... Do we drop the column?

0.111171048316
Blanquette de Volaille et son Riz
Blanquette de Volaille et son Riz
Salade Cesar
Laitue en salade
Danoises à la cannelle roulées
Pâtisserie 
Flute
Flute
Chaussons tressés aux pommes
Pâtisserie
Pain Burger Artisan
Pain
Biscuits sablés fourrage au cacao
Biscuits sablés déclassés fourrage au cacao
Quiche Lorraine
Quiche lorraine
Pâté au poulet
Pâté à la viande
Bonbons acidulés Raisin Fraise
Bonbons acidulés Raisin Fraise
Pâte d'Amandes
Pâte d'Amandes
Pâte à Sucre
Pâte à Sucre - Fondant
Brioches roulées avec raisins
Pâtisserie
เล็บมือนาง (ตีนไก่ )
chicken feet
Belgische Pralinen
Pralinenmischung
Pastille Vichy
Bonbon Vichy avec édulcorants, parfum menthe
M&M's Peanut Butter
M&M's beurre de cacahuète
Pepsi, Nouveau goût !
Boisson gazeuse rafraîchissante aux extraits naturels de végétaux
Chicken Chips
Nuggets de poulet - Chicken Chips
Cauliflower
Cauliflower
Salsa de mostaza
Salsa de mostaza
7Up
Boisson gazeuse aux extraits naturels de citron et de citron vert
Mehrkomponeneten

Farmhouse Oatmeal
Sliced Bread
Milano Distinctive Cookies
Milano Cookies
MILANO DARK CHOCOLATE
GALLETAS CON RELLENO SABOR A CHOCOLATE
STRAWBERRY COOKIES
GALLETAS CON MERMELADA DE FRESA
Farmhouse Sourdough
Sliced bread
MILANO RAPSBERRY FLAVORED CHOCOLATE
GALLETAS CON RELLENO SABOR A CHOCOLATE Y FRAMBUESA
Cinnamon raisin baggel
Cinnamon Raisin Bagels
Tahoe White Chocolate Macadamia
Chocolate Chunk Crispy Cookies
Sausalito Milk Chocolate Macadamia
Chocolate chunk Crispy Cookies
Double chocolate Nantuket Dark Chocolate
Chocolate Chunk Crispy Cookies
Santa Cruz Soft Baked Oatmeal Raisin Soft Cookies
Oatmeal Raisin Cookies
MILANO MINT CHOCOLATE
GALLETAS CON RELLENO SABOR A CHOCOLATE Y MENTA
CHESSMEN COOKIES
GALLETAS CON MERMELADA DE FRESA
Goldfish Colors
Baked snack crackers
Goldfish Baked Snack Crackers Cheddar
Baked Snack Crackers Cheddar
Goldfish Flavor Blasted Xtra Cheddar
Baked Crackers
Goldfish Original Baked Snack Crackers
Baked Snack Crackers - Goldfish Original Baked Snack Crackers


Blueberries
Blueberries
100% Soja Protein Vanille
100% Soja Protein Vanille Geschmack
Piña en Rodajas
Piña en rodajas en su jugo
Purée Tradition aux oignons crus
Coulis de tomates aux oignons crus
Purée de tomates
Purée de tomates Double concentré 28/30%
Fruit Cocktail in juice
Fruit Cocktail
Pear Halves in Juice
Pear Halves in Pineapple Juice
Tomatenstücke
Tomatenstücke in Tomatensaft
Piña en rodajas en zumo
Piña en rodajas en zumo
Tomates Datterini au jus de tomates
Tomates Datterini non pelées au jus de tomates
Pezzi di ananas con sciroppo aromatizzato al limone 
Ananas sciroppata 
Sweet Peas
Sweet peas
Argo Sweet Peas
Petits pois en boite
Tomates pelées au jus Bio
Tomates entières biologiques pelées au jus
Cheez-It
Baked Snack Crackers
Nutty Bars
Wafers with peanut butter
Bat Brownies
brownie
Elotitos tiernos San Miguel
Elotitos tiernos baby corn
Elotitos tiernos San Miguel
Elotitos tiernos
Cous Cous original San Miguel
Cous Cous original
SALSA VERDE GREEN SAUCE
SALSA VERDE
TOMATE 

Italian Tomato Purée double concentrate
Double concentrate tomato purée
Made Without Wheat Top Iced Christmas Cake Bar
Gâteau aux fruits sans gluten avec des raisins secs (33%) et des cerises confites (7%), recouvert de massepain (9%) et de glaçage tendre. - Made Without Wheat Top Iced Christmas Cake Bar
SAUCE TOMATO & BASIL
SALSA DE TOMATE Y ALBAHACA
VODKA SAUCE
SALSA DE TOMATE Y VODKA
Fair-trade Golden Caster
Unrefined Cane Sugar
Joe's O's
Toasted Whole Grain Oats Cereal
Oikos strawberry
Yaourt
Activia light blueberry
Yaourt
Gatorade Sabor Naranja
Bebida isotónica sabor naranja
Gatorade Limonada
Bebida isotónica sabor limonada
Gatorade Lima–Limón
Bebida isotónica sabor Lima–Limón
Gatorade Sabor Uva
Bebida isotónica sabor uva
Gatorade sabor Ponche de Frutas
Bebida in calorías sabor Ponche de frutas con electrolíticos y vitaminas
Hojuelas de avena natural
Avena
Gatorade Fierce Sabor Moras
Bebida isotónica sabor moras
Gatorade Polvo sabor Ponche de Frutas
Polvo sabor Ponche de Frutas ba

4C SEASONED
Pan molido
4C PANKO
Pan molido estilo jaónés
4C WHOLE WHEAT SEASONED
Pan molido
4C PLAIN
Pan molido
SALSA DE SOYA REDUCIDA EN SODIO
SALSA DE SOYA
SALSA DE SOYA TRADICIONAL
KIKKOMAN
Sukiyaki Sauce
sauce teriyaki (french)
TEMPURA EXTRA CRISPY
HARINA DE TRIGO PARA TEMPURA
Organic cereal cinnamon squares
Cereals
Harina completa preparada para hot cakes echos con chispas de chocolate
Harina para hot cakes
Harina completa preparada para hot cakes echos con suero de mantequilla
Harina para hot cakes
Harina preparada para galletas estilo polvorones de canela
Harina para galletas
Harina preparada para pastel vateado con canela
Harina para pastel
Harina preparada para pastelistos sabor mora azul
Harina para pastel
Harina preparada para pastelillos estilo brownies
Harina para pastelillos
Harina preparada para pastelitos sabor plátano con nuez
Harina para pastel
Harina preparada para pastelitos con trozos de chocolate
Harina para pastel
Triple Chocolate Brownie Mix
Préparation pour bro

...

sparkling scottish Moutain watzr
Glen Brae sparkling scottish moutain water
Buttermilk
Buttermilk
Sour Cream
Sour Cream
B12 sublingual tablets
B12 pills
Marks & Spencer   Chicken Saag Masala
Chicken Masala
Canneberges Séchées
Canneberges Séchées
excel gomme a macher
excel
Yogourt ferme Riviera
Yougourt nature
Pêches en dés
Pêches au sirop
Premium Organic Tofu Extra Firm
tofu
Chaire de crabe
Chaire de crabe
Mélange du randonneur
barres granola tendres
Cheerios Miel & Noix
Cheerios Honey & Nut
Multi Grain Cheerios
Céréales multi-grain
Avoine croquante Trio de petits fruits
Oatmeal Crisp
Cheerios a saveur de chocolat
Céréales au chocolat
Granola Thins
Cookies with chocolate coating
Fruitsations
Collation Fruits
Multi-Grain Cheerios
Multi-Grain Cheerios
Bouchées Granola au beurre d'amande
Granola au beurre d'amande en bouchée
Yogourt
Yogourt Liberté méditerrannée
Liberté Grec (0% MG)
Yogourt grec 
Liberté Crème Fraîche
Crème fraîche
Yaourt vanille
Yogourt Vanille
Realemon
Jus de citron
Sir

Carrot Cake
Lightly spiced carrot cake With orange soaked raisins, pineapple, coconut and walnuts, topped With cream cheese frostinq and finished With a cinnamon dusting
Perdue Harvestland Fresh Boneless Skinless Chicken Breasts, Individually Wrapped
Boneless Skinless Chicken Breasts
Anchor Steam
Bière
merci
finest assortment of european chocolates
Haricots blancs
Haricots blancs secs
Diced Beets
Betteraves en dés
Dumpling sauce Hot
Dipping sauce for dumplings, cooked chicken & meat
Swiss knight 
Fromage fondu  tartiner avec gruyèere
Organic Green Tea with Jasmine
Infusion
Calming Teabags
Infusion
Fairtrade Drinking Chocolate
Chocolat en poudre
Honey Wheat Berry Bread
Bread
Pocky almond crush Glico
Galletas de trigo con cubierta sabor a chocolate y almendra
Pocky Almond Crush
Chocolate covered biscuit sticks
Pocky Strawberry
Strawberry Cream Covered Biscuit Sticks
Pocky
Chocolate cream covered biscuit sticks
Mezzetta Sweet Banana Wax Peppers
pickled sweet peppers
Jeff's Naturals Whole 

Egg, Tomato & Salad Cream
free range egg and salad cream with vine ripened tomatoes on malted brown bread
Chinese style noodles
Noodles
Nouilles Impériales Saveur &quot;Soup Wontow&quot;
Mì ăn liền wontow
6 Welsh Cakes
Gâteaux - 6 Welsh Cakes
West Country Crème Fraîche
Crème Fraîche
Coleslaw
Chou, carottes et oignons dans une sauce à la mayonnaise
CapriSun Super V Fruit & Vegetable Juice Drink
Fruit & Vegetable Juice
Rebel IPA
Bière forte
Poudre de Piment Rouge
Red pepper powder
Kimchi / Chou chinois
Chou chinois (pe-tsaï) lacto-fermenté et piments
Grain Bar Rice
Crackers De Riz Coréen
Mirin seasoning
Vin de cuisine sucré
6 pack assorted crisps
Ready salted potato crisps, salt and vinegar flavour potato crisps, cheese and onion flavour potato crisps
Hi-C Pomme
Jus à base de concentré de pomme
Hi-C ORANGE MANGUE
Jus à base de concentré d'orange
Fanta saveur orange
Soda gazéifié
MANGUE
Rotui mangue
Hi-C pomme fraise
jus de pomme aromatisé à la fraise
Jus à base de concentré d'orange (20%

Porc terroir cote filet à griller
Pizza 3 fromages
Pizza
Poulet roti
Poulet roti
Couronne des rois
Brioche
Sainte Maure de Touraine
Sainte Maure de Touraine
poulet roti fermier
poulet roti
Bananes cavendish biologique catégorie 2
bananes
Emmental bio
Fromage à pâte pressée cuite issu de l'agriculture bioologique
gras double de bœuf tripe cuites 
tripes
tablier de sapeur cuit
tablier de sapeur cuit
Dinde farcie
Dinde farcie pomme, figue et marrons


KeyboardInterrupt: 

In [78]:
# We check is the "countries" columns
print(np.sum(food_cleaned.countries_en.isnull())-len(food_cleaned.countries_en))
for i in range(len(food_cleaned.countries_en)):
    if (not (pd.isnull(food_cleaned.countries_en[i])) and not (food_cleaned.countries_en[i]=="France") and not (food_cleaned.countries_en[i]=="United States") and not (food_cleaned.countries_en[i]=="United Kingdom")):
        print(food_cleaned.countries_en[i])
        print(food_cleaned.countries_tags[i])
        print(food_cleaned.countries[i])
        print("===========")
        
# Seeing this, we can remove the the column countries_tags and countries since it doesn't give more infos

-694574
Slovakia
en:slovakia
en:SK
Slovakia
en:slovakia
en:SK
Ireland
en:ireland
en:IE
France,United States
en:france,en:united-states
France,États-Unis
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
France, US
Canada
en:canada
Canada
Canada
en:canada
Canada
Canada
en:canada
Canada
Switzerland
en:switzerland
en:CH
Canada
en:canada
Canada
France,United States
en:france,en:united-states
France,États-Unis
Canada
en:canada
en:CA
Canada
en:canada
Canada
Canada
en:canada
Canada
Peru
en:peru
en:PE
Canada
en:canada
en:CA
Canada
en:canada
Canada
Switzerland
en:switzerland
en:CH
Oman
en:oman
en:OM
Spain
en:spain
en:ES
Germany
en:germany
Deutschland
Slovenia
en:slovenia
en:SI
France,Germany
en:france,en:germany
en:FR, en:de
France,Germany
en:france,en:germany
en:FR, en:de
France,Germany
en:france,en:germany
en:FR, en:de
Italy
en:italy
en:IT
Romania
en:romania
en:RO
Spain
en:spain
España
Australia
en:australia
en:AU
Portugal
en:portuga

France,United States
en:france,en:united-states
en:france, en:united-states
France,United States
en:france,en:united-states
en:france, en:united-states
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
US, en:france
Italy
en:italy
en:IT
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
en:france, en:united-states
Mexico
en:mexico
MEXICO
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
en:france, en:united-states
France,United States
en:france,en:united-states
US,France
France,United States
en:france,en:united-states
US, en:france
Canada
en:canada
en:CA
France,United States
en:france,en:united-states
en:france, en:united-states
Canada
en:canada
en:CA
Canada
en:canada
en:CA
France,United States
en:france,en:united-states
en:france, en:united-states


France,United States
en:france,en:united-states
US, en:france
France,United Kingdom
en:france,en:united-kingdom
France,Royaume-Uni
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
Australia,France
en:australia,en:france
Australia,France
Saint Lucia
en:saint-lucia
en:LC
France,United States
en:france,en:united-states
en:FR, US
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
France,United States
en:france,en:united-states
en:FR, US
France,Germany
en:france,en:germany
en:FR, en:de
Germany
en:germany
Germany
France,Germany
en:france,en:germany
en:france, en:de
France,Germany
en:france,en:germany
en:france, en:de
France,Germany
en:france,en:germany
France, en:de
France,Germany
en:france,en:germany
en:FR, en:de
Switzerland
en:switzerland
en:CH
France,United Kingdom
en:france,en:united-kingdom
France,United Kingdom
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
Colombia
en:colombia
en:CO
France,United States
en:france,en:united-states
France, en

France,United States
en:france,en:united-states
US, en:france
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
France,United Kingdom
en:france,en:united-kingdom
France,Royaume-Uni
France,United Kingdom
en:france,en:united-kingdom
France,Royaume-Uni
Germany
en:germany
Deutschland
Taiwan
en:taiwan
中華民國
France,En
en:france,en:en
en:FR, en:en
France,Germany
en:france,en:germany
en:FR, en:de
France,Hong Kong
en:france,en:hong-kong
nl:Hong kong, en:france
Hong Kong
en:hong-kong
香港
Hong Kong
en:hong-kong
en:HK
France,United States
en:france,en:united-states
US, en:france
Switzerland
en:switzerland
en:CH
France,United Kingdom
en:france,en:united-kingdom
UK,France
Luxembourg
en:luxembourg
en:LU
Australia
en:australia
en:AU
Hong Kong
en:hong-kong
en:HK
United Kingdom,United States
en:united-kingdom,en:united-states
en:GB, US
France,United States
en:france,en:united-states
en:france, en:united-states
Fr

France,Ireland
en:france,en:ireland
France,Irlande
France,Panama
en:france,en:panama
panama, en:france
France,United Kingdom
en:france,en:united-kingdom
en:france, en:united-kingdom
Taiwan
en:taiwan
en:TW
Canada
en:canada
Canada
France,United Kingdom
en:france,en:united-kingdom
France,Royaume-Uni
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
Mexico
en:mexico
MEXICO
Canada
en:canada
en:CA
France,Switzerland,United States
en:france,en:switzerland,en:united-states
Switzerland,United States, en:france
Belgium,France,Switzerland,United States
en:belgium,en:france,en:switzerland,en:united-states
Belgium,Switzerland,United States, en:france
France,Switzerland
en:france,en:switzerland
en:FR, en:ch
French Polynesia
en:french-polynesia
en:PF
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
France,United Kingdom
en:france,en:united-kingdom
en:france, en:united-kingdom
Hong Kong
en:hong-kong
en:HK
Arub

Sweden
en:sweden
Sverige
France,United Kingdom
en:france,en:united-kingdom
France, en:UK
Réunion
en:reunion
en:RE
Germany
en:germany
Deutschland
Mexico
en:mexico
MEXICO
Mexico
en:mexico
MEXICO
France,United Kingdom
en:france,en:united-kingdom
en:france, en:united-kingdom
France,United States
en:france,en:united-states
en:france, en:united-states
France,United Kingdom
en:france,en:united-kingdom
France,United Kingdom
France,United Kingdom
en:france,en:united-kingdom
France,Royaume-Uni
Canada
en:canada
en:CA
France,United States
en:france,en:united-states
US, en:france
France,United States
en:france,en:united-states
France,États-Unis
France,United States
en:france,en:united-states
en:france, en:united-states
France,United Kingdom
en:france,en:united-kingdom
en:france, en:UK
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
Mexico
en:mexico
México
M

KeyboardInterrupt: 

In [36]:
food_cleaned.drop(columns=["countries_tags","countries"],inplace=True)

In [37]:
food_cleaned.head()

,url,creator,created_t,last_modified_t,product_name,generic_name,quantity,packaging,packaging_tags,brands,...,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url
0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-06-15 10:38:00,2018-06-15 10:40:04,Vitória crackers,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,isagoofy,2018-10-13 21:06:14,2018-10-13 21:06:57,Cacao,NaN,130 g,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,kiliweb,2018-08-31 17:53:02,2018-08-31 17:53:06,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,...,"en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:38,2018-11-16 11:56:38,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2018-11-16 11:56:43,2018-11-16 11:56:43,NaN,NaN,NaN,NaN,NaN,NaN,...,"en:empty,en:to-be-completed,en:nutrition-facts...","Empty,To be completed,Nutrition facts to be co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For the name of the countries, we can replace the names as strings into list of strings and change the name of the column as `countries`

In [38]:
food_cleaned.countries_en = food_cleaned.countries_en.str.split(",")
food_cleaned.rename(index = str, columns = {"countries_en":"countries"},inplace = True)

In [39]:
food_cleaned.countries.head() # Now this part is good

0      [France]
1      [France]
2      [France]
3    [Slovakia]
4    [Slovakia]
Name: countries, dtype: object

In [84]:
# We check is the "labels" columns
print(np.sum(food_cleaned.labels_en.isnull())-len(food_cleaned.labels_en))
for i in range(len(food_cleaned.labels_en)):
    if (not (pd.isnull(food_cleaned.labels_en[i]))):
        print(food_cleaned.labels_en[i])
        print(food_cleaned.labels_tags[i])
        print(food_cleaned.labels[i])
        print("===========")

-102047
Made in France
en:made-in-france
Fabriqué en France
fr:Viande Française,Made in France
fr:viande-francaise,en:made-in-france
Viande Française,Fabriqué en France
Contains GMOs
en:contains-gmos
Contient des OGM
Organic,EU Organic,fr:AB Agriculture Biologique
en:organic,en:eu-organic,fr:ab-agriculture-biologique
AB
Kosher,Contains GMOs
en:kosher,en:contains-gmos
Kascher,Contient des OGM
Green Dot
en:green-dot
Point Vert
Contains GMOs
en:contains-gmos
Contient des OGM
Green Dot
en:green-dot
Point Vert
No artificial flavors,Contains GMOs
en:no-artificial-flavors,en:contains-gmos
Sans arômes artificiels,Contient des OGM
Gluten-free
en:gluten-free
fr:Sans gluten
Incorrect data on label
en:incorrect-data-on-label
Incorrect data on label
Not advised for specific people,PDO,Made in France,With sulfites,fr:Déconseillé aux femmes enceintes,fr:Origine France,fr:Triman
en:not-advised-for-specific-people,en:pdo,en:made-in-france,en:with-sulfites,fr:deconseille-aux-femmes-enceintes,fr:origine-

en:organic,en:usda-organic
USDA Organic
Organic
en:organic
en:organic
Organic,USDA Organic
en:organic,en:usda-organic
Organic,USDA Organic
es:no-vegan
es:no-vegan
No vegan
Halal
en:halal
Halal
Organic
en:organic
en:organic
Organic,EU Organic,Gluten-free,fr:AB Agriculture Biologique
en:organic,en:eu-organic,en:gluten-free,fr:ab-agriculture-biologique
Bio,AB,Sans gluten
Organic
en:organic
en:organic
Organic,USDA Organic,Non GMO project
en:organic,en:usda-organic,en:non-gmo-project
usda organic,non GMO project verified,100% organic
No GMOs
en:no-gmos
Non GMO
Kosher,Orthodox Union Kosher
en:kosher,en:orthodox-union-kosher
OU Kosher,Orthodox Union Kosher
Vegetarian,Vegan
en:vegetarian,en:vegan
Vegetarian,Vegan
Low or no fat,Low fat,No cholesterol,No Trans Fat,Labels-for-education
en:low-or-no-fat,en:low-fat,en:no-cholesterol,en:no-trans-fat,en:labels-for-education
Low fat,No cholesterol,0 g Trans Fat,Labels for Education
Low or no fat,Low fat,No cholesterol,No Trans Fat,Labels-for-education

en:contains-gmos
Contient des OGMs
Contains GMOs
en:contains-gmos
Contient des OGMs
Organic,EU Organic,GB-ORG-02
en:organic,en:eu-organic,en:gb-org-02
Gb-org-02
New
en:new
Nouveau
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGM
Gluten-free
en:gluten-free
Sans gluten
Made in France
en:made-in-france
Fabriqué en France
Gluten-free
en:gluten-free
Sans gluten
Made in France
en:made-in-france
en:made-in-france
Kosher,Gluten-free,Made in France
en:kosher,en:gluten-free,en:made-in-france
Fabriqué en France,Kascher,Sans gluten
Made in France
en:made-in-france
en:made-in-france
Organic
en:organic
en:organic
Organic,EU Organic,fr:AB Agriculture Biologique
en:organic,en:eu-organic,fr:ab-agriculture-biologique
Bio,AB
Organic,EU Organic,fr:AB Agriculture Biologique
en:organic,en:eu-organic,fr:ab-agriculture-biologique
AB Agriculture Biologique,Bio européen
Green Dot
en:green-dot
Point Vert
Not advised for specific people,Not recommended for children u

Vegetarian,Vegan,Gluten-free,Non GMO project
en:vegetarian,en:vegan,en:gluten-free,en:non-gmo-project
Non GMO Project Verified,Vegan,Gluten-free
Green Dot
en:green-dot
Point Vert
Low or no sugar,Vegetarian,No preservatives,No artificial colors,No flavors,No milk,No soy,No sugar
en:low-or-no-sugar,en:vegetarian,en:no-preservatives,en:no-artificial-colors,en:no-flavors,en:no-milk,en:no-soy,en:no-sugar
Sans sucre,Végétarien,Sans lait,Sans soja,Sans colorants artificiels,Sans arômes ajoutés,Sans conservateurs
Organic
en:organic
en:organic
Organic
en:organic
en:organic
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGMs
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGMs
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGM
Contains GMOs
en:contains-gmos
Contient des OGM
Kosher,Contains GMOs,Gluten-free
en:kosher,

Green Dot
en:green-dot
Point Vert
Keep-refrigerated,Official-dip-sponsor-of-the-nfl,Official-sponsor-of-the-nfl
en:keep-refrigerated,en:official-dip-sponsor-of-the-nfl,en:official-sponsor-of-the-nfl
Keep Refrigerated,Official dip sponsor of the NFL,Official sponsor of the NFL
Kosher,Organized Kashrut Kosher,Kosher-pareve,Rabbi-m-weissmandl-kosher
en:kosher,en:organized-kashrut-kosher,en:kosher-pareve,en:rabbi-m-weissmandl-kosher
Kosher Pareve,OK Kosher,Rabbi M. Weissmandl Kosher
Organic
en:organic
Bio
Rainforest Alliance
en:rainforest-alliance
Rainforest Alliance
Organic
en:organic
en:organic
Fabrique-en-italie,Riche-en-zinc
en:fabrique-en-italie,en:riche-en-zinc
Fabriqué en Italie,Riche en zinc
Made-with-fresh-cream-garlic
en:made-with-fresh-cream-garlic
made with fresh cream & garlic
Gluten-free
en:gluten-free
Gluten-free
Kosher,Tablet-K Kosher,From-cows-not-treated-with-rbst,Rbst-free
en:kosher,en:tablet-k-kosher,en:from-cows-not-treated-with-rbst,en:rbst-free
Kosher,From cows not t

...

Végétarien
FSC,FSC Mix,Green Dot,fr:certification-europeenne,fr:fsc-c114413,fr:marquage-ce
en:fsc,en:fsc-mix,en:green-dot,fr:certification-europeenne,fr:fsc-c114413,fr:marquage-ce
Point Vert,Marquage CE,Certification Européenne,FSC Mix,FSC C114413
Not advised for specific people,Green Dot,fr:Déconseillé aux femmes enceintes
en:not-advised-for-specific-people,en:green-dot,fr:deconseille-aux-femmes-enceintes
Déconseillé à certaines catégories de personnes,Point Vert,Déconseillé aux femmes enceintes
Deconseille-aux-femmes-enceintes,Deconseille-a-certaines-categories-de-personnes,Point-vert,Vegetarien
en:deconseille-aux-femmes-enceintes,en:deconseille-a-certaines-categories-de-personnes,en:point-vert,en:vegetarien
Végétarien,Déconseillé à certaines catégories de personnes,Point Vert,Déconseillé aux femmes enceintes
Deconseille-aux-femmes-enceintes,Point-vert
en:deconseille-aux-femmes-enceintes,en:point-vert
Déconseillé aux femmes enceintes,Point Vert
Contient-des-sulfites,Deconseille-aux-f

Green Dot
en:green-dot
Point Vert
Huile-sun-kernel,Vegetarien
en:huile-sun-kernel,en:vegetarien
Huile Sun Kernel,Végétarien
Vegetarien
en:vegetarien
Végétarien
Vegetarian,Gluten-free
en:vegetarian,en:gluten-free
sans gluten,Végétarien
Organic
en:organic
Organic
es:no-vegan
es:no-vegan
no vegan
Kosher,Orthodox Union Kosher
en:kosher,en:orthodox-union-kosher
Kosher,Orthodox Union Kosher
Vegetarian,No artificial flavors,No artificial colors,No artificial colours or flavours,No artificial preservatives,No artificial sweeteners,No GMOs,No hydrogenated fats
en:vegetarian,en:no-artificial-flavors,en:no-artificial-colors,en:no-artificial-colours-or-flavours,en:no-artificial-preservatives,en:no-artificial-sweeteners,en:no-gmos,en:no-hydrogenated-fats
Vegetarian,No artificial colours or flavours,No artificial sweeteners,No artificial preservatives,No hydrogenated fats,No GMOs
Vegetarian
en:vegetarian
Végétarien
Vegetarian,No-hydrogenated-oil,Non-gm-ingredients
en:vegetarian,en:no-hydrogenated-oi

No colorings,No preservatives
en:no-colorings,en:no-preservatives
Sans conservateur,Sans colorant
Incorrect data on label,Incorrect nutrition facts on label,No Trans Fat
en:incorrect-data-on-label,en:incorrect-nutrition-facts-on-label,en:no-trans-fat
No Trans Fat,Incorrect nutrition facts on label
FSC,FSC Mix,fr:fabrique-au-royaume-uni,fr:peut-etre-congele
en:fsc,en:fsc-mix,fr:fabrique-au-royaume-uni,fr:peut-etre-congele
FSC Mix,Fabriqué au Royaume-Uni,Peut-être congelé
Halal
en:halal
Halal
Kosher,Kosher-parve
en:kosher,en:kosher-parve
Kascher pareve
Kosher,Orthodox Union Kosher
en:kosher,en:orthodox-union-kosher
Kosher,Orthodox Union Kosher
Green Dot
en:green-dot
en:green-dot
Wisconsin-master-s-mark
en:wisconsin-master-s-mark
Wisconsin Master's Mark
Gluten-free
en:gluten-free
Gluten-free
Gluten-free
en:gluten-free
Gluten Free
Acs-2012-winner,American-cheese-society-2012-winner,Kascher
en:acs-2012-winner,en:american-cheese-society-2012-winner,en:kascher
ACS 2012 Winner,American Cheese 

en:green-dot
Point Vert
Vegetarian,Green Dot
en:vegetarian,en:green-dot
Végétarien,Point Vert
Chili,Hot,Naga
en:chili,en:hot,en:naga
hot,chili,naga
Green Dot
en:green-dot
Point Vert
Green Dot,fr:peut-etre-congele
en:green-dot,fr:peut-etre-congele
Point Vert,Peut être congelé
Green Dot
en:green-dot
Point Vert
Free range,Free range eggs,Made with free range eggs
en:free-range,en:free-range-eggs,en:made-with-free-range-eggs
Fait avec des œufs de poule en liberté
No artificial flavour enhancer
en:no-artificial-flavour-enhancer
Sin potenciador artificial del sabor
Halal
en:halal
Halal
Halal
en:halal
Halal
Halal,ISO 22000,ISO 9001,Fssai-fssl-10012021000063
en:halal,en:iso-22000,en:iso-9001,en:fssai-fssl-10012021000063
Halal,FSSAI FSSL 10012021000063,ISO 22000,ISO 9001
Halal
en:halal
Halal
Halal
en:halal
Halal
Halal
en:halal
Halal
Green Dot
en:green-dot
Point Vert
Halal
en:halal
Halal
Halal,Coconut-milk-included,Export-product,Lppom-00060000421298
en:halal,en:coconut-milk-included,en:export-p

KeyboardInterrupt: 